In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - test_likelihood
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Do some simple tests of the likelihood functionality of the density 
fitting routines.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, dill as pickle, warnings

## Matplotlib
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

## Project-specific
sys.path.insert(0,'../../../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Scale parameters
ro = 8.275
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup

%matplotlib inline
plt.style.use('../../../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries

In [ ]:
# Pathing
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX']
data_dir_base,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max \
    = putil.parse_config_dict(cdict,keywords)
data_dir = data_dir_base+'gaia_apogee/'
gaia_apogee_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'

### Selection function and effective selection function grid

In [ ]:
# Filenames
apogee_SF_filename = data_dir+gaia_apogee_dir+'apogee_SF.dat'
apogee_effSF_filename = data_dir+gaia_apogee_dir+'apogee_effSF_grid_inclArea_z0.001_logAge10.0.dat'
apogee_effSF_mask_filename = data_dir+gaia_apogee_dir+'apogee_effSF_grid_mask_z0.001_logAge10.0.npy'

# Selection function
print('APOGEE data release is: '+apogee_dr+', and results version is: '+apogee_results_vers)
if os.path.exists(apogee_SF_filename):
    print('Loading APOGEE selection function from '+apogee_SF_filename)
    with open(apogee_SF_filename, 'rb') as f:
        apogee_SF = pickle.load(f)
else:
    sys.exit('Could not find APOGEE selection function, make it. Exiting...')

# Load the effective selection function grid
if os.path.exists(apogee_effSF_filename):
    print('\nLoading effective selection function from '+apogee_effSF_filename)
    with open(apogee_effSF_filename,'rb') as f:
        apogee_effSF_grid_inclArea = pickle.load(f)
else:
    sys.exit('\nFailed to load effective survey section function grid')
    
# Load the effecive selection function mask
if os.path.exists(apogee_effSF_mask_filename):
    print('\nLoading effective selection function mask from '+apogee_effSF_mask_filename)
    apogee_effSF_mask = np.load(apogee_effSF_mask_filename)
else:
    sys.exit('\nFailed to load effective survey section function grid mask')

In [ ]:
# Distance modulus grid
dmods,ds = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=ro,zo=zo)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

# Include the distance factor in the selection function
apof *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
#ds_for_delta = 10**(np.append(dmods-(dmods[1]-dmods[0]),dmods[-1]+(dmods[1]-dmods[0]))/5.-2.)
#apof = apof * ds**2. * np.diff(ds_for_delta)

## Single Power Law
Parameters should be $\alpha=2.5$
rmin is 2 kpc

In [ ]:
# Load data
mock_number = '19'
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')
allstar = allstar[omask]
orbs = orbs[omask]
data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

In [ ]:
n_sample = 100
model = pdens.spherical
alpha_range = np.linspace(0.05,5.,num=n_sample)
alpha_correct = 2.5
likelihood = np.zeros_like(alpha_range)

for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

print('Using mock '+mock_number)
fig.show()

### Single power law
Parameters should be: $\alpha=2.0$

rmin is 1kpc

In [ ]:
# Load data
mock_number = '13'
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')
allstar = allstar[omask]
orbs = orbs[omask]
data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

In [ ]:
n_sample = 100
model = pdens.spherical
alpha_range = np.linspace(0.05,5.,num=n_sample)
alpha_correct = 2.0
likelihood = np.zeros_like(alpha_range)

for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

print('Using mock '+mock_number)
fig.show()

In [ ]:
# Load data
mock_number = '17'
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

allstar = allstar[omask]
orbs = orbs[omask]

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

In [ ]:
n_sample = 100
model = pdens.spherical
alpha_range = np.linspace(0.05,5.,num=n_sample)
alpha_correct = 2.0
likelihood = np.zeros_like(alpha_range)

for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

print('Using mock '+mock_number)
fig.show()

## Single Power Law
Parameters should be: $\alpha=2.0$

rmin is 0.1 kpc

In [ ]:
# Load data
mock_number = '14'
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

allstar = allstar[omask]
orbs = orbs[omask]

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

In [ ]:
n_sample = 100
model = pdens.spherical
alpha_range = np.linspace(0.05,5.,num=n_sample)
alpha_correct = 2.0
likelihood = np.zeros_like(alpha_range)

for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)
    
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

print('Using mock '+mock_number)
fig.show()

### Single Power Law

Parameters should be $\alpha=3.5$

rmin is 0.1 kpc

In [ ]:
# Load data
mock_number = '15'
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

allstar = allstar[omask]
orbs = orbs[omask]

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

In [ ]:
n_sample = 100
model = pdens.spherical
alpha_range = np.linspace(0.05,5.,num=n_sample)
alpha_correct = 3.5
likelihood = np.zeros_like(alpha_range)

for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')

print('Using mock '+mock_number)
fig.show()

## Flattened Single Power Law
Parameters should be: $\alpha=2.5$ and $c=0.5$

rmin is 0.1 kpc

In [ ]:
# Load data
mock_number = '16'
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

allstar = allstar[omask]
orbs = orbs[omask]

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

In [ ]:
n_sample = 100
model = pdens.axisymmetric

alpha_correct = 2.5
c_correct = 0.5
alpha_range = np.linspace(0.05,5.,num=n_sample)
alpha_likelihood = np.zeros_like(alpha_range)

for i in range(len(alpha_range)):
    alpha_likelihood[i] = pmass.mloglike([alpha_range[i],c_correct], model, 
                                          apof, Rgrid, phigrid, zgrid, 
                                          data_R, data_phi, data_z)

c_range = np.linspace(0.,1.,num=n_sample)
c_likelihood = np.zeros_like(c_range)
for i in range(len(c_range)):
    c_likelihood[i] = pmass.mloglike([alpha_correct,c_range[i]], model,
                                      apof, Rgrid, phigrid, zgrid,
                                      data_R, data_phi, data_z)

fig = plt.figure(figsize=(10,4))
axs = fig.subplots(nrows=1, ncols=2)

axs[0].plot(alpha_range, alpha_likelihood)
axs[0].axvline(alpha_range[np.argmin(alpha_likelihood)], linestyle='dashed')
axs[0].axvline(alpha_correct, linestyle='dashed', color='DarkOrange')
axs[0].set_xlabel(r'$\alpha$')
axs[0].set_ylabel(r'$\log \mathcal{L}$')

axs[1].plot(c_range, c_likelihood)
axs[1].axvline(c_range[np.argmin(c_likelihood)], linestyle='dashed')
axs[1].axvline(c_correct, linestyle='dashed', color='DarkOrange')
axs[1].set_xlabel(r'$c$')
axs[1].set_ylabel(r'$\log \mathcal{L}$')

print('Using mock '+mock_number)
fig.show()

## Triaxial Power Law
Parameters should be: $\alpha=3.5$, $b=0.5$, $c=0.7$

rmin is 0.5 kpc

In [ ]:
# Load data
mock_number = '18'
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

allstar = allstar[omask]
orbs = orbs[omask]

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

In [ ]:
n_sample = 100
model = pdens.triaxial_norot

alpha_correct = 3.5
b_correct = 0.5
c_correct = 0.7

alpha_range = np.linspace(0.05,5.,num=n_sample)
alpha_likelihood = np.zeros_like(alpha_range)
for i in range(len(alpha_range)):
    alpha_likelihood[i] = pmass.mloglike([alpha_range[i],b_correct,c_correct], 
                                          model, apof, Rgrid, phigrid, zgrid, 
                                          data_R, data_phi, data_z)

b_range = np.linspace(0.,1.,num=n_sample)
b_likelihood = np.zeros_like(b_range)
for i in range(len(b_range)):
    b_likelihood[i] = pmass.mloglike([alpha_correct,b_range[i],c_correct], 
                                      model, apof, Rgrid, phigrid, zgrid,
                                      data_R, data_phi, data_z)
    
c_range = np.linspace(0.,1.,num=n_sample)
c_likelihood = np.zeros_like(c_range)
for i in range(len(c_range)):
    c_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_range[i]], 
                                      model, apof, Rgrid, phigrid, zgrid,
                                      data_R, data_phi, data_z)

fig = plt.figure(figsize=(15,4))
axs = fig.subplots(nrows=1, ncols=3)

axs[0].plot(alpha_range, alpha_likelihood)
axs[0].axvline(alpha_range[np.argmin(alpha_likelihood)], linestyle='dashed')
axs[0].axvline(alpha_correct, linestyle='dashed', color='DarkOrange')
axs[0].set_xlabel(r'$\alpha$')
axs[0].set_ylabel(r'$\log \mathcal{L}$')

axs[1].plot(c_range, c_likelihood)
axs[1].axvline(b_range[np.argmin(b_likelihood)], linestyle='dashed')
axs[1].axvline(b_correct, linestyle='dashed', color='DarkOrange')
axs[1].set_xlabel(r'$b$')
axs[1].set_ylabel(r'$\log \mathcal{L}$')

axs[2].plot(c_range, c_likelihood)
axs[2].axvline(c_range[np.argmin(c_likelihood)], linestyle='dashed')
axs[2].axvline(c_correct, linestyle='dashed', color='DarkOrange')
axs[2].set_xlabel(r'$c$')
axs[2].set_ylabel(r'$\log \mathcal{L}$')

print('Using mock '+mock_number)
fig.show()

## Modify rmin
Show likelihood curves for many different values of rmin

In [ ]:
mock_rmin = [2.0,1.0,0.5,0.1]
mock_numbers = ['20','21','22','23']
mock_colors = ['Red','ForestGreen','DodgerBlue','Purple']
alpha_correct = 2.5
model = pdens.spherical
n_sample = 100

fig = plt.figure()
ax = fig.add_subplot(111)

for i in range(len(mock_rmin)):
    # Data
    with open('../../data/mock_'+mock_numbers[i]+'/orbs.pkl','rb') as f:
        orbs = pickle.load(f)
    omask = np.load('../../data/mock_'+mock_numbers[i]+'/omask.npy')
    orbs = orbs[omask]
    data_R = orbs.R().value
    data_phi = orbs.phi().value
    data_z = orbs.z().value
    
    alpha_range = np.linspace(2.,3.,num=n_sample)
    alpha_likelihood = np.zeros_like(alpha_range)
    for j in range(len(alpha_range)):
        alpha_likelihood[j] = pmass.mloglike([alpha_range[j],], 
                                              model, apof, Rgrid, phigrid, zgrid, 
                                              data_R, data_phi, data_z)
    alpha_likelihood = alpha_likelihood/np.min(alpha_likelihood)
    
    print(len(data_R))
    
    
    ax.plot(alpha_range, alpha_likelihood, color=mock_colors[i], alpha=0.5)
    ax.axvline(alpha_range[np.argmin(alpha_likelihood)], color=mock_colors[i],
               linestyle='dashed', label='rmin: '+str(mock_rmin[i]))
    
ax.axvline(alpha_correct, linestyle='solid', color='Black')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'Normalized $\log \mathcal{L}$')

fig.show()

In [ ]:
mock_rmin = [2.0,1.5,1.0,0.5,0.1]
mock_numbers = ['24','25','26','27','28']
mock_colors = ['Red','DarkOrange','ForestGreen','DodgerBlue','Purple']
alpha_correct = 2.5
model = pdens.spherical
n_sample = 100

fig = plt.figure()
ax = fig.add_subplot(111)

for i in range(len(mock_rmin)):
    # Data
    orbs_filename = '../../data/mock_'+mock_numbers[i]+'/orbs.pkl'
    with open(orbs_filename,'rb') as f:
        orbs = pickle.load(f)
    omask_filename = '../../data/mock_'+mock_numbers[i]+'/omask.npy'
    omask = np.load(omask_filename)
    print(orbs_filename)
    print(omask_filename)
    orbs = orbs[omask]
    data_R = orbs.R().value
    data_phi = orbs.phi().value
    data_z = orbs.z().value
    
    alpha_range = np.linspace(2.,3.,num=n_sample)
    alpha_likelihood = np.zeros_like(alpha_range)
    for j in range(len(alpha_range)):
        alpha_likelihood[j] = pmass.mloglike([alpha_range[j],], 
                                              model, apof, Rgrid, phigrid, zgrid, 
                                              data_R, data_phi, data_z)
    alpha_likelihood = alpha_likelihood/np.min(alpha_likelihood)
    
    print(len(data_R))
    
    ax.plot(alpha_range, alpha_likelihood, color=mock_colors[i], alpha=0.5)
    ax.axvline(alpha_range[np.argmin(alpha_likelihood)], color=mock_colors[i],
               linestyle='dashed', label='rmin: '+str(mock_rmin[i]))
    
ax.axvline(alpha_correct, linestyle='solid', color='Black')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'Normalized $\log \mathcal{L}$')

fig.show()

In [ ]:
mock_rmin = [2.0,1.5,1.0,0.5,0.1]
mock_numbers = ['29','30','31','32','33']
mock_colors = ['Red','DarkOrange','ForestGreen','DodgerBlue','Purple']
alpha_correct = 3.5
model = pdens.spherical
n_sample = 100

fig = plt.figure()
ax = fig.add_subplot(111)

for i in range(len(mock_rmin)):
    
    # if i > 2: continue
    
    # Data
    orbs_filename = '../../data/mock_'+mock_numbers[i]+'/orbs.pkl'
    with open(orbs_filename,'rb') as f:
        orbs = pickle.load(f)
    omask_filename = '../../data/mock_'+mock_numbers[i]+'/omask.npy'
    omask = np.load(omask_filename)
    print(orbs_filename)
    print(omask_filename)
    orbs = orbs[omask]
    data_R = orbs.R().value
    data_phi = orbs.phi().value
    data_z = orbs.z().value
    
    alpha_range = np.linspace(3.,4.,num=n_sample)
    alpha_likelihood = np.zeros_like(alpha_range)
    for j in range(len(alpha_range)):
        alpha_likelihood[j] = pmass.mloglike([alpha_range[j],], 
                                              model, apof, Rgrid, phigrid, zgrid, 
                                              data_R, data_phi, data_z)
    alpha_likelihood = alpha_likelihood/np.min(alpha_likelihood)
    
    print(len(data_R))
    
    ax.plot(alpha_range, alpha_likelihood, color=mock_colors[i], alpha=0.5)
    ax.axvline(alpha_range[np.argmin(alpha_likelihood)], color=mock_colors[i],
               linestyle='dashed', label='rmin: '+str(mock_rmin[i]))
    
ax.axvline(alpha_correct, linestyle='solid', color='Black')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'Normalized $\log \mathcal{L}$')

fig.show()

In [ ]:
mock_rmin = [4.0,2.0,1.5,1.0,0.5,0.1]
#mock_numbers = ['29','30','31','32','33']
mock_colors = ['Black','Red','DarkOrange','ForestGreen','DodgerBlue','Purple']
alpha_correct = 3.5
model = pdens.spherical
n_sample = 100

fig = plt.figure()
ax = fig.add_subplot(111)

# Data
mock_number = '33'
orbs_filename = '../../data/mock_'+mock_number+'/orbs.pkl'
with open(orbs_filename,'rb') as f:
    orbs = pickle.load(f)
omask_filename = '../../data/mock_'+mock_number+'/omask.npy'
omask = np.load(omask_filename)

for i in range(len(mock_rmin)):
        
    rmin_omask = orbs.r(use_physical=True).value > mock_rmin[i]
    
    print('length of rmin_omask '+str(np.sum(rmin_omask)))
    
    orbs_mask = orbs[omask&rmin_omask]
    data_R = orbs_mask.R().value
    data_phi = orbs_mask.phi().value
    data_z = orbs_mask.z().value
    
    alpha_range = np.linspace(3.,4.,num=n_sample)
    alpha_likelihood = np.zeros_like(alpha_range)
    for j in range(len(alpha_range)):
        alpha_likelihood[j] = pmass.mloglike([alpha_range[j],], 
                                              model, apof, Rgrid, phigrid, zgrid, 
                                              data_R, data_phi, data_z)
    alpha_likelihood = alpha_likelihood/np.min(alpha_likelihood)
    
    print(len(data_R))
    
    ax.plot(alpha_range, alpha_likelihood, color=mock_colors[i], alpha=0.5)
    ax.axvline(alpha_range[np.argmin(alpha_likelihood)], color=mock_colors[i],
               linestyle='dashed', label='rmin: '+str(mock_rmin[i]))
    
ax.axvline(alpha_correct, linestyle='solid', color='Black')
ax.legend()
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'Normalized $\log \mathcal{L}$')

fig.show()

### Take a look at some triaxial profiles to see how the program evaluates the likelihood

In [ ]:
eta,theta=pdens.zvec_to_eta_theta([1.,0.,0.])
eta_norm = (eta+1)/2
theta_norm = theta/(2*np.pi)
print(eta)
print(theta)
print(eta_norm)
print(theta_norm)

In [ ]:
eta,theta=pdens.zvec_to_eta_theta([0,1.,0.])
eta_norm = (eta+1)/2
theta_norm = theta/(2*np.pi)
print(eta)
print(theta)
print(eta_norm)
print(theta_norm)

In [ ]:
eta,theta=pdens.zvec_to_eta_theta([0.707,0.,0.707])
eta_norm = (eta+1)/2
theta_norm = theta/(2*np.pi)
print(eta)
print(theta)
print(eta_norm)
print(theta_norm)

In [ ]:
mock_numbers = ['34','35','36','37','38','39']

# [alpha,b,c,theta,eta,pa]
# theta normalized 0 to 2pi
# eta normalized -1 to 1
# pa normalized 0 to pi
params_correct = [
    [3.5, 0.5, 1.0, 0.0  , 1.0  , 0.0 ],
    [3.5, 0.5, 1.0, 0.0  , 1.0  , 0.25],
    [3.5, 0.5, 1.0, 0.0  , 1.0  , 0.5 ],
    [3.5, 0.5, 1.0, 0.0  , 0.5  , 0.0 ],
    [3.5, 0.5, 1.0, 0.25 , 0.5  , 0.0 ],
    [3.5, 0.5, 1.0, 0.0  , 0.854, 0.25]
]


for k in range(len(mock_numbers)):

    # Load data
    mock_number = mock_numbers[k]
    allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
    with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
        orbs = pickle.load(f)
    ##wi
    omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

    allstar = allstar[omask]
    orbs = orbs[omask]

    data_R = orbs.R().value
    data_phi = orbs.phi().value
    data_z = orbs.z().value

    n_sample = 100
    model = pdens.triaxial_single_angle_zvecpa
    
    alpha_correct,b_correct,c_correct,theta_correct,eta_correct,pa_correct =\
        params_correct[k]
    print(pdens.eta_theta_to_zvec(eta_correct,theta_correct))

    print('alpha: '+str(alpha_correct))
    print('b: '+str(b_correct))
    print('c: '+str(c_correct))
    print('theta: '+str(theta_correct))
    print('eta: '+str(eta_correct))
    print('pa: '+str(pa_correct))


    alpha_range = np.linspace(0.05,5.,num=n_sample)
    alpha_likelihood = np.zeros_like(alpha_range)
    for i in range(len(alpha_range)):
        alpha_likelihood[i] = pmass.mloglike([alpha_range[i],b_correct,c_correct,
                                              theta_correct,eta_correct,pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid, 
                                              data_R, data_phi, data_z)

    b_range = np.linspace(0.,1.,num=n_sample)
    b_likelihood = np.zeros_like(b_range)
    for i in range(len(b_range)):
        b_likelihood[i] = pmass.mloglike([alpha_correct,b_range[i],c_correct,
                                          theta_correct,eta_correct,pa_correct], 
                                          model, apof, Rgrid, phigrid, zgrid,
                                          data_R, data_phi, data_z)

    c_range = np.linspace(0.,1.,num=n_sample)
    c_likelihood = np.zeros_like(c_range)
    for i in range(len(c_range)):
        c_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_range[i],
                                          theta_correct,eta_correct,pa_correct], 
                                          model, apof, Rgrid, phigrid, zgrid,
                                          data_R, data_phi, data_z)

    theta_range = np.linspace(0.,1.,num=n_sample)
    theta_likelihood = np.zeros_like(c_range)
    for i in range(len(c_range)):
        theta_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                              theta_range[i],eta_correct,pa_correct], 
                                          model, apof, Rgrid, phigrid, zgrid,
                                          data_R, data_phi, data_z)

    eta_range = np.linspace(0.,1.,num=n_sample)
    eta_likelihood = np.zeros_like(c_range)
    for i in range(len(c_range)):
        eta_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                              theta_correct,eta_range[i],pa_correct], 
                                          model, apof, Rgrid, phigrid, zgrid,
                                          data_R, data_phi, data_z)

    pa_range = np.linspace(0.,1.,num=n_sample)
    pa_likelihood = np.zeros_like(c_range)
    for i in range(len(c_range)):
        pa_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                              theta_correct,eta_correct,pa_range[i]], 
                                          model, apof, Rgrid, phigrid, zgrid,
                                          data_R, data_phi, data_z)    

    fig = plt.figure(figsize=(15,8))
    axs = fig.subplots(nrows=2, ncols=3).flatten()

    axs[0].plot(alpha_range, alpha_likelihood)
    axs[0].axvline(alpha_range[np.argmin(alpha_likelihood)], linestyle='dashed')
    axs[0].axvline(alpha_correct, linestyle='dashed', color='DarkOrange')
    axs[0].set_xlabel(r'$\alpha$')
    axs[0].set_ylabel(r'$\log \mathcal{L}$')

    axs[1].plot(b_range, b_likelihood)
    axs[1].axvline(b_range[np.argmin(b_likelihood)], linestyle='dashed')
    axs[1].axvline(b_correct, linestyle='dashed', color='DarkOrange')
    axs[1].set_xlabel(r'$b$')
    axs[1].set_ylabel(r'$\log \mathcal{L}$')

    axs[2].plot(c_range, c_likelihood)
    axs[2].axvline(c_range[np.argmin(c_likelihood)], linestyle='dashed')
    axs[2].axvline(c_correct, linestyle='dashed', color='DarkOrange')
    axs[2].set_xlabel(r'$c$')
    axs[2].set_ylabel(r'$\log \mathcal{L}$')

    axs[3].plot(theta_range, theta_likelihood)
    axs[3].axvline(theta_range[np.argmin(theta_likelihood)], linestyle='dashed')
    axs[3].axvline(theta_correct, linestyle='dashed', color='DarkOrange')
    axs[3].set_xlabel(r'$\theta$')
    axs[3].set_ylabel(r'$\log \mathcal{L}$')
    ax_theta = axs[3].twinx()
    zvec_theta = pdens.eta_theta_to_zvec(eta_correct*np.pi, theta_range)
    ax_theta.plot(theta_range, zvec_theta[0], color='Red')
    ax_theta.plot(theta_range, zvec_theta[1], color='ForestGreen')
    ax_theta.plot(theta_range, zvec_theta[2], color='Navy')
    ax_theta.set_ylabel(r'$\hat{Z}_{i}$')
    ax_theta.set_ylim(-0.05,1.05)

    axs[4].plot(eta_range, eta_likelihood)
    axs[4].axvline(eta_range[np.argmin(eta_likelihood)], linestyle='dashed')
    axs[4].axvline(eta_correct, linestyle='dashed', color='DarkOrange')
    axs[4].set_xlabel(r'$\eta$')
    axs[4].set_ylabel(r'$\log \mathcal{L}$')
    ax_eta = axs[4].twinx()
    zvec_eta = pdens.eta_theta_to_zvec(eta_range, theta_correct*np.pi)
    ax_eta.plot(eta_range, zvec_eta[0], color='Red')
    ax_eta.plot(eta_range, zvec_eta[1], color='ForestGreen')
    ax_eta.plot(eta_range, zvec_eta[2], color='Navy')
    ax_eta.set_ylabel(r'$\hat{Z}_{i}$')
    ax_eta.set_ylim(-0.05,1.05)

    axs[5].plot(pa_range, pa_likelihood)
    axs[5].axvline(pa_range[np.argmin(pa_likelihood)], linestyle='dashed')
    axs[5].axvline(pa_correct, linestyle='dashed', color='DarkOrange')
    axs[5].set_xlabel(r'$\phi$')
    axs[5].set_ylabel(r'$\log \mathcal{L}$')

    print('Using mock '+mock_number)
    fig.tight_layout()
    fig.show()



In [ ]:
def normalize_theta(theta,denorm=False):
    if denorm:
        return theta*2*np.pi
    else:
        return theta/(2*np.pi)

def normalize_eta(eta,denorm=False):
    if denorm:
        return 2*eta-1
    else:
        return (eta+1)/2
    
def normalize_pa(pa,denorm=False):
    if denorm:
        return pa*np.pi
    else:
        return pa/np.pi

In [ ]:
pdens.zvec_to_eta_theta([0.5,0.,0.5])

In [ ]:
mock_numbers = ['40','41','42','43','44','45']

# [alpha,b,c,theta,eta,pa]
# theta normalized 0 to 2pi
# eta normalized -1 to 1
# pa normalized 0 to pi
params_correct = [
    [3.5, 0.5, 1.0, 0.0      , 1.0  , np.pi/6],
    [3.5, 0.5, 0.8, 0.0      , 1.0  , np.pi/5],
    [3.5, 1.0, 0.5, 0.0      , 1.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, np.pi/2. , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.707, np.pi/6]
]

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for k in range(len(mock_numbers)):

        # Load data
        mock_number = mock_numbers[k]
        print('\nUsing mock '+str(mock_number))
        allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
        with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
            orbs = pickle.load(f)
        ##wi
        omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

        allstar = allstar[omask]
        orbs = orbs[omask]

        data_R = orbs.R().value
        data_phi = orbs.phi().value
        data_z = orbs.z().value

        n_sample = 100
        model = pdens.triaxial_single_angle_zvecpa

        alpha_correct,b_correct,c_correct,theta_correct,eta_correct,pa_correct =\
            params_correct[k]
        
        print(pdens.eta_theta_to_zvec(eta_correct,theta_correct))
        print('alpha: '+str(alpha_correct))
        print('b: '+str(b_correct))
        print('c: '+str(c_correct))
        print('theta: '+str(theta_correct))
        print('eta: '+str(eta_correct))
        print('pa: '+str(pa_correct))
        
        # Normalize for density input
        eta_correct = normalize_eta(eta_correct)
        theta_correct = normalize_theta(theta_correct)
        pa_correct = normalize_pa(pa_correct)

        alpha_range = np.linspace(0.05,5.,num=n_sample)
        alpha_likelihood = np.zeros_like(alpha_range)
        for i in range(len(alpha_range)):
            alpha_likelihood[i] = pmass.mloglike([alpha_range[i],b_correct,c_correct,
                                                  theta_correct,eta_correct,pa_correct], 
                                                  model, apof, Rgrid, phigrid, zgrid, 
                                                  data_R, data_phi, data_z)

        b_range = np.linspace(1e-4,0.9999,num=n_sample)
        b_likelihood = np.zeros_like(b_range)
        for i in range(len(b_range)):
            b_likelihood[i] = pmass.mloglike([alpha_correct,b_range[i],c_correct,
                                              theta_correct,eta_correct,pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)

        c_range = np.linspace(1e-4,0.9999,num=n_sample)
        c_likelihood = np.zeros_like(c_range)
        for i in range(len(c_range)):
            c_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_range[i],
                                              theta_correct,eta_correct,pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)

        theta_range = np.linspace(1e-4,0.9999,num=n_sample)
        theta_likelihood = np.zeros_like(c_range)
        for i in range(len(c_range)):
            theta_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                                  theta_range[i],eta_correct,pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)

        eta_range = np.linspace(1e-4,0.9999,num=n_sample)
        eta_likelihood = np.zeros_like(c_range)
        for i in range(len(c_range)):
            eta_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                                theta_correct,eta_range[i],pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)

        pa_range = np.linspace(1e-4,0.9999,num=n_sample)
        pa_likelihood = np.zeros_like(c_range)
        for i in range(len(c_range)):
            pa_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                                  theta_correct,eta_correct,pa_range[i]], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)    

        fig = plt.figure(figsize=(15,8))
        fig.suptitle('Mock '+str(mock_number))
        axs = fig.subplots(nrows=2, ncols=3).flatten()

        axs[0].plot(alpha_range, alpha_likelihood)
        axs[0].axvline(alpha_range[np.argmin(alpha_likelihood)], linestyle='dashed', 
                       label=r'Min. $\mathcal{L}$', zorder=2, linewidth=3)
        axs[0].axvline(alpha_correct, linestyle='solid', color='DarkOrange', 
                       label='correct', zorder=1, linewidth=3)
        axs[0].set_xlabel(r'$\alpha$')
        axs[0].set_ylabel(r'$\log \mathcal{L}$')

        axs[1].plot(b_range, b_likelihood)
        axs[1].axvline(b_range[np.argmin(b_likelihood)], linestyle='dashed', 
                       zorder=2, linewidth=3)
        axs[1].axvline(b_correct, linestyle='solid', color='DarkOrange', 
                       zorder=1, linewidth=3)
        axs[1].set_xlabel(r'$b$')
        axs[1].set_ylabel(r'$\log \mathcal{L}$')

        axs[2].plot(c_range, c_likelihood)
        axs[2].axvline(c_range[np.argmin(c_likelihood)], linestyle='dashed', 
                       zorder=2, linewidth=3)
        axs[2].axvline(c_correct, linestyle='solid', color='DarkOrange',
                       zorder=1, linewidth=3)
        axs[2].set_xlabel(r'$c$')
        axs[2].set_ylabel(r'$\log \mathcal{L}$')

        axs[3].plot(theta_range, theta_likelihood)
        axs[3].axvline(theta_range[np.argmin(theta_likelihood)], linestyle='dashed',
                       zorder=2, linewidth=3)
        axs[3].axvline(theta_correct, linestyle='solid', color='DarkOrange', 
                       zorder=1, linewidth=3)
        axs[3].set_xlabel(r'$\theta$')
        axs[3].set_ylabel(r'$\log \mathcal{L}$')
        ax_theta = axs[3].twinx()
        zvec_theta = pdens.eta_theta_to_zvec(eta_correct*np.pi, theta_range)
        ax_theta.plot(theta_range, zvec_theta[0], color='Red', alpha=0.25)
        ax_theta.plot(theta_range, zvec_theta[1], color='ForestGreen', alpha=0.25)
        ax_theta.plot(theta_range, zvec_theta[2], color='Navy', alpha=0.25)
        ax_theta.set_ylabel(r'$\hat{Z}_{i}$')
        ax_theta.set_ylim(-0.05,1.05)

        axs[4].plot(eta_range, eta_likelihood)
        axs[4].axvline(eta_range[np.argmin(eta_likelihood)], linestyle='dashed',
                       zorder=2, linewidth=3)
        axs[4].axvline(eta_correct, linestyle='solid', color='DarkOrange',
                       zorder=1, linewidth=3)
        axs[4].set_xlabel(r'$\eta$')
        axs[4].set_ylabel(r'$\log \mathcal{L}$')
        ax_eta = axs[4].twinx()
        zvec_eta = pdens.eta_theta_to_zvec(eta_range, theta_correct*np.pi)
        ax_eta.plot(eta_range, zvec_eta[0], color='Red', alpha=0.25)
        ax_eta.plot(eta_range, zvec_eta[1], color='ForestGreen', alpha=0.25)
        ax_eta.plot(eta_range, zvec_eta[2], color='Navy', alpha=0.25)
        ax_eta.set_ylabel(r'$\hat{Z}_{i}$')
        ax_eta.set_ylim(-0.05,1.05)

        axs[5].plot(pa_range, pa_likelihood)
        axs[5].axvline(pa_range[np.argmin(pa_likelihood)], linestyle='dashed', 
                       zorder=2, linewidth=3)
        axs[5].axvline(pa_correct, linestyle='solid', color='DarkOrange', 
                       zorder=1, linewidth=3)
        axs[5].set_xlabel(r'$\phi$')
        axs[5].set_ylabel(r'$\log \mathcal{L}$')

        fig.tight_layout()
        fig.show()



### Try changing `ro` to 8 and `zo` to 0 for the eSSF grid

In [ ]:
# Distance modulus grid
dmods,ds = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=8,zo=0)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

# Include the distance factor in the selection function
apof *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
#ds_for_delta = 10**(np.append(dmods-(dmods[1]-dmods[0]),dmods[-1]+(dmods[1]-dmods[0]))/5.-2.)
#apof = apof * ds**2. * np.diff(ds_for_delta)

mock_numbers = ['40','41','42','43','44','45']

# [alpha,b,c,theta,eta,pa]
# theta normalized 0 to 2pi
# eta normalized -1 to 1
# pa normalized 0 to pi
params_correct = [
    [3.5, 0.5, 1.0, 0.0      , 1.0  , np.pi/6],
    [3.5, 0.5, 0.8, 0.0      , 1.0  , np.pi/5],
    [3.5, 1.0, 0.5, 0.0      , 1.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, np.pi/2. , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.707, np.pi/6]
]

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for k in range(len(mock_numbers)):

        # Load data
        mock_number = mock_numbers[k]
        print('\nUsing mock '+str(mock_number))
        allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
        with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
            orbs = pickle.load(f)
        ##wi
        omask = np.load('../../data/mock_'+mock_number+'/omask.npy')

        allstar = allstar[omask]
        orbs = orbs[omask]

        data_R = orbs.R().value
        data_phi = orbs.phi().value
        data_z = orbs.z().value

        n_sample = 100
        model = pdens.triaxial_single_angle_zvecpa

        alpha_correct,b_correct,c_correct,theta_correct,eta_correct,pa_correct =\
            params_correct[k]
        
        print(pdens.eta_theta_to_zvec(eta_correct,theta_correct))
        print('alpha: '+str(alpha_correct))
        print('b: '+str(b_correct))
        print('c: '+str(c_correct))
        print('theta: '+str(theta_correct))
        print('eta: '+str(eta_correct))
        print('pa: '+str(pa_correct))
        
        # Normalize for density input
        eta_correct = normalize_eta(eta_correct)
        theta_correct = normalize_theta(theta_correct)
        pa_correct = normalize_pa(pa_correct)

        alpha_range = np.linspace(0.05,5.,num=n_sample)
        alpha_likelihood = np.zeros_like(alpha_range)
        for i in range(len(alpha_range)):
            alpha_likelihood[i] = pmass.mloglike([alpha_range[i],b_correct,c_correct,
                                                  theta_correct,eta_correct,pa_correct], 
                                                  model, apof, Rgrid, phigrid, zgrid, 
                                                  data_R, data_phi, data_z)

        b_range = np.linspace(1e-4,0.9999,num=n_sample)
        b_likelihood = np.zeros_like(b_range)
        for i in range(len(b_range)):
            b_likelihood[i] = pmass.mloglike([alpha_correct,b_range[i],c_correct,
                                              theta_correct,eta_correct,pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)

        c_range = np.linspace(1e-4,0.9999,num=n_sample)
        c_likelihood = np.zeros_like(c_range)
        for i in range(len(c_range)):
            c_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_range[i],
                                              theta_correct,eta_correct,pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)

        theta_range = np.linspace(1e-4,0.9999,num=n_sample)
        theta_likelihood = np.zeros_like(c_range)
        for i in range(len(c_range)):
            theta_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                                  theta_range[i],eta_correct,pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)

        eta_range = np.linspace(1e-4,0.9999,num=n_sample)
        eta_likelihood = np.zeros_like(c_range)
        for i in range(len(c_range)):
            eta_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                                theta_correct,eta_range[i],pa_correct], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)

        pa_range = np.linspace(1e-4,0.9999,num=n_sample)
        pa_likelihood = np.zeros_like(c_range)
        for i in range(len(c_range)):
            pa_likelihood[i] = pmass.mloglike([alpha_correct,b_correct,c_correct,
                                                  theta_correct,eta_correct,pa_range[i]], 
                                              model, apof, Rgrid, phigrid, zgrid,
                                              data_R, data_phi, data_z)    

        fig = plt.figure(figsize=(15,8))
        fig.suptitle('Mock '+str(mock_number))
        axs = fig.subplots(nrows=2, ncols=3).flatten()

        axs[0].plot(alpha_range, alpha_likelihood)
        axs[0].axvline(alpha_range[np.argmin(alpha_likelihood)], linestyle='dashed', 
                       label=r'Min. $\mathcal{L}$', zorder=2, linewidth=3)
        axs[0].axvline(alpha_correct, linestyle='solid', color='DarkOrange', 
                       label='correct', zorder=1, linewidth=3)
        axs[0].set_xlabel(r'$\alpha$')
        axs[0].set_ylabel(r'$\log \mathcal{L}$')

        axs[1].plot(b_range, b_likelihood)
        axs[1].axvline(b_range[np.argmin(b_likelihood)], linestyle='dashed', 
                       zorder=2, linewidth=3)
        axs[1].axvline(b_correct, linestyle='solid', color='DarkOrange', 
                       zorder=1, linewidth=3)
        axs[1].set_xlabel(r'$b$')
        axs[1].set_ylabel(r'$\log \mathcal{L}$')

        axs[2].plot(c_range, c_likelihood)
        axs[2].axvline(c_range[np.argmin(c_likelihood)], linestyle='dashed', 
                       zorder=2, linewidth=3)
        axs[2].axvline(c_correct, linestyle='solid', color='DarkOrange',
                       zorder=1, linewidth=3)
        axs[2].set_xlabel(r'$c$')
        axs[2].set_ylabel(r'$\log \mathcal{L}$')

        axs[3].plot(theta_range, theta_likelihood)
        axs[3].axvline(theta_range[np.argmin(theta_likelihood)], linestyle='dashed',
                       zorder=2, linewidth=3)
        axs[3].axvline(theta_correct, linestyle='solid', color='DarkOrange', 
                       zorder=1, linewidth=3)
        axs[3].set_xlabel(r'$\theta$')
        axs[3].set_ylabel(r'$\log \mathcal{L}$')
        ax_theta = axs[3].twinx()
        zvec_theta = pdens.eta_theta_to_zvec(eta_correct*np.pi, theta_range)
        ax_theta.plot(theta_range, zvec_theta[0], color='Red', alpha=0.25)
        ax_theta.plot(theta_range, zvec_theta[1], color='ForestGreen', alpha=0.25)
        ax_theta.plot(theta_range, zvec_theta[2], color='Navy', alpha=0.25)
        ax_theta.set_ylabel(r'$\hat{Z}_{i}$')
        ax_theta.set_ylim(-0.05,1.05)

        axs[4].plot(eta_range, eta_likelihood)
        axs[4].axvline(eta_range[np.argmin(eta_likelihood)], linestyle='dashed',
                       zorder=2, linewidth=3)
        axs[4].axvline(eta_correct, linestyle='solid', color='DarkOrange',
                       zorder=1, linewidth=3)
        axs[4].set_xlabel(r'$\eta$')
        axs[4].set_ylabel(r'$\log \mathcal{L}$')
        ax_eta = axs[4].twinx()
        zvec_eta = pdens.eta_theta_to_zvec(eta_range, theta_correct*np.pi)
        ax_eta.plot(eta_range, zvec_eta[0], color='Red', alpha=0.25)
        ax_eta.plot(eta_range, zvec_eta[1], color='ForestGreen', alpha=0.25)
        ax_eta.plot(eta_range, zvec_eta[2], color='Navy', alpha=0.25)
        ax_eta.set_ylabel(r'$\hat{Z}_{i}$')
        ax_eta.set_ylim(-0.05,1.05)

        axs[5].plot(pa_range, pa_likelihood)
        axs[5].axvline(pa_range[np.argmin(pa_likelihood)], linestyle='dashed', 
                       zorder=2, linewidth=3)
        axs[5].axvline(pa_correct, linestyle='solid', color='DarkOrange', 
                       zorder=1, linewidth=3)
        axs[5].set_xlabel(r'$\phi$')
        axs[5].set_ylabel(r'$\log \mathcal{L}$')

        fig.tight_layout()
        fig.show()

# Distance modulus grid
dmods,ds = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=ro,zo=zo)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

# Include the distance factor in the selection function
apof *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
#ds_for_delta = 10**(np.append(dmods-(dmods[1]-dmods[0]),dmods[-1]+(dmods[1]-dmods[0]))/5.-2.)
#apof = apof * ds**2. * np.diff(ds_for_delta)

### Do a couple spherical power laws

In [ ]:
# Params
mock_number = '46'
n_sample = 100
model = pdens.spherical
alpha_correct = 3.5

# Load data
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')
allstar = allstar[omask]
orbs = orbs[omask]
print('Number of orbits: '+str(len(orbs)))

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

alpha_range = np.linspace(1.,4.,num=n_sample)
likelihood = np.zeros_like(alpha_range)
for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', 
           label=r'Min $\mathcal{L}$')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange', 
           label='Truth')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')
ax.legend()

print('Using mock '+mock_number)
fig.show()

In [ ]:
# Params
mock_number = '47'
n_sample = 100
model = pdens.spherical
alpha_correct = 3.5

# Load data
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')
allstar = allstar[omask]
orbs = orbs[omask]
print('Number of orbits: '+str(len(orbs)))

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

alpha_range = np.linspace(1.,4.,num=n_sample)
likelihood = np.zeros_like(alpha_range)
for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', 
           label=r'Min $\mathcal{L}$')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange', 
           label='Truth')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')
ax.legend()

print('Using mock '+mock_number)
fig.show()

In [ ]:
# Params
mock_number = '48'
n_sample = 100
model = pdens.spherical
alpha_correct = 2.5

# Load data
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')
allstar = allstar[omask]
orbs = orbs[omask]
print('Number of orbits: '+str(len(orbs)))

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

alpha_range = np.linspace(1.,4.,num=n_sample)
likelihood = np.zeros_like(alpha_range)
for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', 
           label=r'Min $\mathcal{L}$')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange', 
           label='Truth')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')
ax.legend()

print('Using mock '+mock_number)
fig.show()

In [ ]:
# Params
mock_number = '49'
n_sample = 200
model = pdens.spherical
alpha_correct = 2.5

# Load data
allstar = np.load('../../data/mock_'+mock_number+'/allstar.npy')
with open('../../data/mock_'+mock_number+'/orbs.pkl','rb') as f:
    orbs = pickle.load(f)
##wi
omask = np.load('../../data/mock_'+mock_number+'/omask.npy')
allstar = allstar[omask]
orbs = orbs[omask]
print('Number of orbits: '+str(len(orbs)))

data_R = orbs.R().value
data_phi = orbs.phi().value
data_z = orbs.z().value

alpha_range = np.linspace(1.,4.,num=n_sample)
likelihood = np.zeros_like(alpha_range)
for i in range(len(alpha_range)):
    likelihood[i] = pmass.mloglike([alpha_range[i]], model, apof,
                                   Rgrid, phigrid, zgrid, 
                                   data_R, data_phi, data_z)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(alpha_range, likelihood)
ax.axvline(alpha_range[np.argmin(likelihood)], linestyle='dashed', 
           label=r'Min $\mathcal{L}$')
ax.axvline(alpha_correct, linestyle='dashed', color='DarkOrange', 
           label='Truth')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel(r'$\log \mathcal{L}$')
ax.legend()

print('Using mock '+mock_number)
fig.show()